# Scenario Query Examples

This notebook demonstrates how to use `scripts/query_scenarios.py` for exploratory analysis.


In [ ]:
import sys

sys.path.insert(0, "..")

from scripts.query_scenarios import ScenarioQuery, quick_query, compare_params

## Example 1: List available variables and parameters


In [ ]:
query = ScenarioQuery("../data_parquet")

print("Available variables:")
for var in query.list_variables():
    print(f"  - {var}")

print("\nAvailable parameters:")
for param in query.param_cols:
    values = query.scenarios.get_column(param).unique().sort().to_list()
    print(f"  - {param}: {values}")

## Example 2: Filter scenarios by constraints


In [ ]:
# Fix Fertility Variation and Climate scenario, see remaining combinations
filtered = query.filter_scenarios(
    {"Fertility Variation": 1.6, "Climate change scenario switch for water yield": 1}
)

print(f"Matching scenarios: {filtered.height}")
print(f"Columns: {filtered.columns}")
filtered.head()

## Example 3: Get time series under constraints


In [ ]:
data = quick_query(
    variable="Total population",
    filters={"Fertility Variation": 1.6},
    data_dir="../data_parquet",
    time_range=(2020, 2050),
)

print(f"Shape: {data.shape}")
data.head()

## Example 4: Compare parameter impact (wide pivot for easy plotting)


In [ ]:
# Fix all params except irrigation efficiency, pivot on it for comparison
comparison = compare_params(
    variable="YRB WSI",
    fixed_params={
        "Fertility Variation": 1.6,
        "fire generation share province target": 0.1,
        "Ecological water flow variable": 0.2,
        "Climate change scenario switch for water yield": 1,
        "Diet change scenario switch": 1,
    },
    vary_param="water-saving irrigation efficiency ratio",
    data_dir="../data_parquet",
    time_range=(2020, 2100),
)

print(f"Shape: {comparison.shape}")
comparison.head()

## Example 5: Using safe variable names (original ↔ safe)

Safe variable names are generated for stable file/column keys:
- Lowercase; non-alphanumerics replaced by `_`; multiple `_` collapsed.
- Mapping file: `data_parquet/variables_map.json` (original → safe).
- You can query by original names (human-friendly) or safe names (stable keys).


In [ ]:
import json
import pathlib

map_path = pathlib.Path("../data_parquet/variables_map.json")
variables_map = json.loads(map_path.read_text()) if map_path.exists() else {}
variables_map

In [ ]:
# Query by original name (human-friendly)
q = ScenarioQuery("../data_parquet")
orig = quick_query(
    variable="hydrologic station discharge[lijin]",
    filters={"Fertility Variation": 1.6},
    data_dir="../data_parquet",
    time_range=(2020, 2020),
)
orig.select(["scenario_name", "variable", "time", "value"]).head()

In [ ]:
# Query by safe name (stable key)
safe = quick_query(
    variable="hydrologic_station_discharge_lijin",
    filters={"Fertility Variation": 1.6},
    data_dir="../data_parquet",
    time_range=(2020, 2020),
)
safe.select(["scenario_name", "variable", "time", "value"]).head()